<a href="https://colab.research.google.com/github/Kusanagi242/BTVN/blob/main/TGM_BIRD_DECTECT_Nh%C3%B3m_%C4%90%E1%BB%A9c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Trần Minh Đức - 18146105
#Nguyễn Trần Toàn Phú - 18146189
#Phan Xuân Lộc - 18146171

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') #Connect to google drive

In [ ]:
!ls gdrive/MyDrive/

In [ ]:
!ls gdrive/MyDrive/images/


In [ ]:
#preparing libary
import numpy as np
import functools
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


train_root = Path('gdrive/MyDrive/images/') #Tạo đường dẫn 

In [ ]:
#Khởi tạo các thông số cơ bản cho data đầu vào
batch_size = 18
img_height = 224 #Resize lại các ảnh data đầu vào
img_width = 224   #Resize lại các ảnh data đầu vào

#Tạo tập Training
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_root, #đường dẫn đầu vào
    validation_split=0.2, #Use 80% data to training
    subset="training",
    seed = 123,
    image_size = (img_height,img_width),
    batch_size = batch_size
)

In [ ]:
#Tạp tập Validation
val_ds = tf.keras.utils.image_dataset_from_directory(
    train_root,
    validation_split=0.2, #Use 20% data to validation
    subset="validation",
    seed = 123,
    image_size = (img_height,img_width),
    batch_size = batch_size
)

In [ ]:
#create matrix
a = np.eye(200)
#create dataset
filelist_ds = tf.data.Dataset.list_files(str(train_root/'*/*'))

In [ ]:
#Tạo Label cho các file
def get_label(file_path):
  for item in train_root.glob("*"):
    if item.name[0] == '0': #0xx
      if item.name[1]== '0': #00x
        i = item.name[2],     # i =x
        i = functools.reduce(lambda sub, ele: sub * 10 + ele, i) # change tuple to str
        i = int(i)-1 #change str to int  #Do giá trị đầu tiên của ma trận là 0 nên giá trị đầu của tệp là 1 -1 để về đúng ma trận 0 
        labels = a[i]
      else: #0ab with a 0->9  
        i = item.name[1] + item.name[2], 
        i = functools.reduce(lambda sub, ele: sub * 10 + ele, i)
        i = int(i)-1
        labels = a[i]
    if item.name[0] == '1': #1xx
      if item.name[1] == '0': #10x
        i = str(1) +str(0)+ item.name[2],
        i = functools.reduce(lambda sub, ele: sub * 10 + ele, i)
        i = int(i)-1
        labels = a[i]
      else: #1ab with a 0->9
        i = str(1) + item.name[1] + item.name[2],
        i = functools.reduce(lambda sub, ele: sub * 10 + ele, i) 
        i = int(i)-1
        labels = a[i]
    if item.name[0]=='2': #2xx
      if item.name[1] == '0':
        i = str(2) +str(0)+ item.name[2],
        i = functools.reduce(lambda sub, ele: sub * 10 + ele, i)
        i = int(i)-1
        labels = a[i]
  return tf.convert_to_tensor(labels) #Error: Chưa xác định được 

In [ ]:
def preprocessing(file_path):
  # read file
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3) #channels =3 --> ảnh màu, 1--> xám
  # transform
  # convert from uint8 to float32 and normalize values to [0, 1]
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize image
  img = tf.image.resize(img, [img_width, img_height])
  # .......
  # get image label
  label = get_label(file_path)

  # return
  return img, label

In [ ]:
ds_size = 6027
batch_size = 18
# Build dataset
train_ds = filelist_ds.shuffle(ds_size)
train_ds = train_ds.map(preprocessing, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(1)

In [ ]:
#create model with Sequential API
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(224,224,3))) #Input Values
#1 ///////////////////////////////////////////////////////
model.add(layers.Conv2D(64,3,strides=(1,1),padding= 'same', activation='relu'))
model.add(layers.Conv2D(64,3,strides=(1,1),padding= 'same', activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
#2 ///////////////////////////////////////////////////////
model.add(layers.Conv2D(128,3,strides=(1,1),padding= 'same', activation='relu'))
model.add(layers.Conv2D(128,3,strides=(1,1),padding= 'same', activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
#3 ///////////////////////////////////////////////////////
model.add(layers.Conv2D(256,3,strides=(1,1),padding= 'same', activation='relu'))
model.add(layers.Conv2D(256,3,strides=(1,1),padding= 'same', activation='relu'))
model.add(layers.Conv2D(256,3,strides=(1,1),padding= 'same', activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
#4 ///////////////////////////////////////////////////////
model.add(layers.Conv2D(512,3,strides=(1,1),padding= 'same', activation='relu'))
model.add(layers.Conv2D(512,3,strides=(1,1),padding= 'same', activation='relu'))
model.add(layers.Conv2D(512,3,strides=(1,1),padding= 'same', activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
#5 ///////////////////////////////////////////////////////
model.add(layers.Conv2D(512,3,strides=(1,1),padding= 'same', activation='relu'))
model.add(layers.Conv2D(512,3,strides=(1,1),padding= 'same', activation='relu'))
model.add(layers.Conv2D(512,3,strides=(1,1),padding= 'same', activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
#model.add(layers.Dropout(1))
model.add(layers.Dense(4096, activation='relu'))
#model.add(layers.Dropout(1))
model.add(layers.Dense(200, activation='softmax'))


In [ ]:
model.summary() #Xem thong tin cac lop

In [ ]:
(X_train,Y_train),(X_test,Y_test) = mnist.load_data()
#print(X_train.shape)

In [ ]:
#reshape and normalize
X_train = X_train.reshape((X_train.shape[0],28,28,1)).astype('float32')/255 #Chuyển biến số nguyên thành số thực để normalize
X_test = X_test.reshape((X_test.shape[0],28,28,1)).astype('float32')/255

Y_train = to_categorical(Y_train,200)
Y_test = to_categorical(Y_test,200)


In [ ]:
# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

In [ ]:
# Train model
model.fit(train_ds, epochs=10)

In [ ]:
model.save('gdrive/MyDrive/Model_Trained/BirdTrained.hdf5')